In [ ]:
import os
import math
import json
import pickle
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow import keras
from keras_vggface.vggface import VGGFace

from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.applications import MobileNet, ResNet50, InceptionV3
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from keras_vggface.utils import preprocess_input as vggface_preprocess
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import Callback, LearningRateScheduler
from tensorflow_addons.optimizers import CyclicalLearningRate
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda, \
Conv1D, Attention, GlobalAveragePooling1D, BatchNormalization, Concatenate, \
Layer, Reshape
from inception_resnet_v1 import InceptionResNetV1
from keras_facenet import FaceNet

random.seed(123)
tf.random.set_seed(12)
np.random.seed(123)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
train_path = './data/train'

# Training pairs generating

Available training pairs from csv files are splitted to train - validation sets. Those pairs are positive(there is blood relation). For each set(train/valid) we additionally generate negative pairs.

Positive pairs are generated according to the input csv file. For each person of positive pair we create one negative pair.
In total we'll have twice more negative than positive pairs.

In [ ]:
def make_image_pair(pair, input_shape, shuffle=True, slice_imgs=1):
    '''
    Create pair of embeddings.
    
    Arguments:
    p1, p2 -- paths to persons' images directories (familyID/personID)
    
    Returns:
    pairs -- array of image pairs, pairing is alligned to smaller number of images
    ''' 
    p1, p2 = [os.path.join(train_path, p) for p in pair]
    
    p1_imgs = os.listdir(p1)
    p2_imgs = os.listdir(p2)
    
    if shuffle:
        random.shuffle(p1_imgs)
        random.shuffle(p2_imgs)
    
    p1_imgs = p1_imgs[:slice_imgs]
    p2_imgs = p2_imgs[:slice_imgs]
    
    for i in range(len(p1_imgs)):
        for j in range(len(p2_imgs)):
            img1_path = os.path.join(p1, p1_imgs[i])
            img2_path = os.path.join(p2, p2_imgs[j])
            img1 = image.load_img(img1_path, target_size=(input_shape[0], input_shape[1]))
            img2 = image.load_img(img2_path, target_size=(input_shape[0], input_shape[1]))
            img1 = np.array(img1).astype('float32')
            img2 = np.array(img2).astype('float32')
            
            yield img1, img2

In [ ]:
def pairs_set(input_pairs, input_shape, shuffle=True, slice_imgs=1):
    for pair, label in input_pairs:
        try:
            emb_pairs = make_image_pair(pair, input_shape, shuffle, slice_imgs)
            for emb_pair in emb_pairs:
                yield emb_pair, label
        except (KeyError, FileNotFoundError):
            continue

def batched_pairs(input_pairs, batch_size, dataset_period, input_shape, preprocess, shuffle=True, slice_imgs=1):
    imgs1 = []
    imgs2 = []
    labels = []
    counter = 0
    for example in pairs_set(input_pairs, input_shape, shuffle, slice_imgs):
        # Get every nth sample
        counter += 1
        if counter % dataset_period:
            continue
        
        exmpls, label = example
        exmpl1, exmpl2 = exmpls
        imgs1.append(exmpl1)
        imgs2.append(exmpl2)
        labels.append(label)
        if len(labels) == batch_size:
            yield {'input_1':preprocess(np.array(imgs1)), 'input_2':preprocess(np.array(imgs2))}, np.array(labels).astype(float)
            imgs1, imgs2, labels = [], [], []

In [ ]:
with open('train_val_set.json', 'r') as f:
    train_val_set = json.load(f)

train_rlt_list, neg_train_rltshps, valid_rlt_list, neg_valid_rltshps = list(train_val_set.values())
train_rlt_list = train_rlt_list * 4

train_rlts = list(zip(train_rlt_list + neg_train_rltshps, [True]*len(train_rlt_list) + [False]*len(neg_train_rltshps)))
val_rlts = list(zip(valid_rlt_list + neg_valid_rltshps, [True]*len(valid_rlt_list) + [False]*len(neg_valid_rltshps)))

random.shuffle(train_rlts)
random.shuffle(val_rlts)

# Siamese network

Initial experimenting is done with conv1D deep neural network, as additional option for experimenting there is simple attention module.

In [ ]:
def mobilenet(input_shape, l2_value, dropout):
    mobile = MobileNet(
        input_shape=input_shape,
        dropout=dropout,
        include_top=False,
        pooling='avg',
        alpha=1.,
        weights='imagenet'
    )
    
    for layer in mobile.layers:
        layer.trainable = True
        if hasattr(layer, 'kernel_regularizer'):
            setattr(layer, 'kernel_regularizer', keras.regularizers.l2(l2_value))
        
    x = Dense(512, kernel_regularizer=l2(l2_value), activation='relu')(mobile.output)
    x = Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
    return Model(mobile.input, x)

def inception(input_shape, l2_value, dropout):
    inception = InceptionV3(
        input_shape=input_shape,
        include_top=False,
        pooling='avg',
        weights='imagenet'
    )
    
    for layer in inception.layers:
        layer.trainable = True
        if hasattr(layer, 'kernel_regularizer'):
            setattr(layer, 'kernel_regularizer', keras.regularizers.l2(l2_value))
        
    x = Dense(128, kernel_regularizer=l2(l2_value), activation='relu')(inception.output)
    x = Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
    return Model(inception.input, x)

def resnet50(input_shape, l2_value, dropout):
    resnet = ResNet50(
        input_shape=input_shape,
        include_top=False,
        pooling='avg',
        weights='imagenet'
    )
    
    for layer in resnet.layers:
        layer.trainable = True
        if hasattr(layer, 'kernel_regularizer'):
            setattr(layer, 'kernel_regularizer', keras.regularizers.l2(l2_value))
        
    x = Dense(32, kernel_regularizer=l2(l2_value), activation='relu')(resnet.output)
    x = Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
    return Model(resnet.input, x)

def vggface_resnet50(input_shape, l2_value, dropout, model_name='model'):
    vggface_res = VGGFace(model='resnet50', include_top=False, input_shape=input_shape)
    
    first_half = True
    for layer in vggface_res.layers:
        first_half = first_half and layer.name != 'add_10'
        layer.trainable = first_half
        if hasattr(layer, 'kernel_regularizer'):
            setattr(layer, 'kernel_regularizer', keras.regularizers.l2(l2_value))
    
    last_layer = vggface_res.get_layer('avg_pool').output
    x = Reshape((-1, 1))(last_layer)
    
    x_1 = Conv1D(1, 128)(x)
    x_1 = Conv1D(1, 128)(x_1)
    x_1 = Flatten()(x_1)
    x_1 = Dense(2048, kernel_regularizer=l2(l2_value), activation='relu')(x_1)
    x_1 = Lambda(lambda x: K.l2_normalize(x,axis=1))(x_1)
    
#     x_2 = Conv1D(1, 128)(x)
#     x_2 = Conv1D(1, 128)(x_2)
#     x_2 = Flatten()(x_2)
#     x_2 = Dense(2048, kernel_regularizer=l2(l2_value), activation='relu')(x_2)
#     x_2 = Lambda(lambda x: K.l2_normalize(x,axis=1))(x_2)
    
    return Model(vggface_res.input, x_1, name=model_name)

def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

def facenet_inception_resnetv1(input_shape, l2_value, dropout, model_name='model'):
    inception_resnet = InceptionResNetV1(input_shape=input_shape, weights_path='keras-facenet/weights/facenet_keras_weights.h5')
    
    first_half = True
    for layer in inception_resnet.layers:
        first_half = first_half and layer.name != 'add_10'
        layer.trainable = first_half
        if hasattr(layer, 'kernel_regularizer'):
            setattr(layer, 'kernel_regularizer', keras.regularizers.l2(l2_value))
    
    return inception_resnet

# Loss and metrics functions

In [ ]:
MARGIN = 0.7

# Margins for positive and negative pairs in the batch
margin_pos = 0.6 * MARGIN
margin_neg = 1.4 * MARGIN

def euclidean_distance(vectors):
    x, y = vectors
    sum_square = K.sum(K.square(x - y), axis=1)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def cosine_distance(vectors):
    x, y = vectors
    x_norm = tf.norm(x, axis=1)
    y_norm = tf.norm(y, axis=1)
    x_y_dot = tf.einsum('ij,ij->i', x, y)
    cos_sim = x_y_dot / (x_norm * y_norm + K.epsilon())
    return 1. - cos_sim

def cos_euc_distance(vectors):
    euc = euclidean_distance(vectors)
    cos_dist = cosine_distance(vectors)
    return cos_dist * euc

def cross_correlation(vectors):
    '''
    The goal is to convolute outputs from both networks,
    each one from the batch of the first network output
    over appropriate one from another network output
    '''
    
    # x and y have shape: [batch_size, vector_size]
    x, y = vectors
    # We need to add 'channels' dimension -> [batch_size, vector_size, 1]
    x = tf.expand_dims(x, -1)
    y = tf.expand_dims(y, -1)
    
    # Firstly we do convolution as the second network
    # output vectors from the batch are all filters.
    # Technically, we've done convolution of all y over each x.
    # From documentation(https://www.tensorflow.org/api_docs/python/tf/nn/conv1d) desired shapes should be:
    # x shape: batch_shape(batch_size) + [in_width(vector_size), in_channels(1)]
    # y shape: [filter_width(vector_size), in_channels(1), out_channels(batch_size)]
    # Output shape is [batch_size, vector_length, batch_size].
    conv1d = tf.nn.conv1d(x, tf.transpose(y, perm=[1, 2, 0]), stride=1, padding='SAME')
    conv1d_shape = conv1d.shape
    
    # We need now to apply mask to the output in order to get desired
    # convolution results in the shape [batch_size, vector_length]
    bool_diag = tf.linalg.diag(tf.constant([True] * batch_size))
    mask = tf.repeat(tf.expand_dims(bool_diag, axis=1), conv1d_shape[1], axis=1)
    out = tf.squeeze(tf.ragged.boolean_mask(conv1d, mask), axis=-1)
    return 1. - tf.reduce_max(out, 1)

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    weight_pos = 1.6
    weight_neg = 1.
    
    pred_pos = tf.boolean_mask(y_pred, y_true)
    pred_neg = tf.boolean_mask(y_pred, 1 - y_true)
    pred_pos_m = tf.reduce_mean(pred_pos, axis=0)
    pred_neg_m = tf.reduce_mean(pred_neg, axis=0)
    var_pos = 0. if tf.equal(tf.size(pred_pos), 0) else tf.reduce_mean(tf.square(pred_pos_m - pred_pos))
    var_neg = 0. if tf.equal(tf.size(pred_neg), 0) else tf.reduce_mean(tf.square(pred_neg_m - pred_neg))
    variance_loss = 0.5 * (var_pos + var_neg)
    
    square_pos = tf.square(tf.maximum(pred_pos - margin_pos, 0))
    square_neg = tf.square(tf.maximum(margin_neg - pred_neg, 0))
    squared_concat = tf.concat([weight_pos * square_pos, weight_neg * square_neg], axis=0)
    return tf.reduce_mean(squared_concat) + 1.0 * variance_loss

# Run training

In [ ]:
input_shape = (160, 160, 3)
learning_rate = 7e-7
l2_value = 1e-8
dropout = 0.
epochs = 1000
batch_size = 16
dataset_period = 1
eval_dataset_period = 1
model_type = 'facenet_inception_resnetv1'
preprocess = prewhiten
# 'euclidean_distance', 'cosine_distance', 'cos_euc_distance'
distance_type = 'cosine_distance'
optimizer = 'Adam'

# Learning rate scheduler
def scheduler(epoch, lr):
    if epoch == 10:
        return 0.5 * lr
    elif epoch == 180:
        return 0.8 * lr
    elif epoch == 250:
        return 0.5 * lr
    elif epoch == 300:
        return 0.7 * lr
    elif epoch == 400:
        return 0.3 *lr
    elif epoch == 700:
        return 0.6 *lr
    return lr
    
lr_callback = LearningRateScheduler(scheduler)

# Create dictionary of parameters for saving configuration
train_config = {}
for name in [
    'learning_rate',
    'l2_value',
    'dropout',
    'epochs',
    'batch_size',
    'model_type',
    'dataset_period',
    'eval_dataset_period',
    'distance_type',
    'optimizer',
    'MARGIN'
]:
    train_config[name] = eval(name)

In [ ]:
data_augmentation = keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.2, width_factor=0.2),
    keras.layers.experimental.preprocessing.RandomContrast(factor=0.2),
    keras.layers.experimental.preprocessing.RandomZoom(height_factor=0.2)
])

In [ ]:
base_network = eval(model_type)(input_shape, l2_value, dropout, 'base_network')
base_network.count_params()

In [ ]:
# Creation of Siamese network
input1 = Input(shape=input_shape, name='input_1')
input2 = Input(shape=input_shape, name='input_2')

input_1_aug = data_augmentation(input1)
input_2_aug = data_augmentation(input2)

processed1 = base_network(input_1_aug)
processed2 = base_network(input_2_aug)

In [ ]:
dist_function = eval(distance_type)
distance = Lambda(dist_function,
                  output_shape=eucl_dist_output_shape)([processed1, processed2])

model = Model([input1, input2], distance)

## Run tensorboard plugin in order to track changes of training

In [ ]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir=./logs --port=7008

## Training Callbacks

In [ ]:
# Get training set length
train_len = 0
for pair, label in train_rlts:
    try:
        p1, p2 = [os.path.join(train_path, p) for p in pair]
        p1_imgs = os.listdir(p1)
        p2_imgs = os.listdir(p2)
        train_len += 1
    except FileNotFoundError:
        continue

train_len = train_len // dataset_period

val_len = 0
for pair, label in val_rlts:
    try:
        p1, p2 = [os.path.join(train_path, p) for p in pair]
        p1_imgs = os.listdir(p1)
        p2_imgs = os.listdir(p2)
        val_len += 2*2
    except FileNotFoundError:
        continue

val_len = val_len // eval_dataset_period

print(f'Train set length: {train_len}')
print(f'Valid set length: {val_len}')

In [ ]:
def val_distance_stats(predictions, labels):
    val_pos = predictions[labels.astype(np.bool)]
    val_neg = predictions[(1 - labels).astype(np.bool)]
    val_pos_m, val_pos_s = np.mean(val_pos), np.std(val_pos)
    val_neg_m, val_neg_s = np.mean(val_neg), np.std(val_neg)
    
    return val_pos_m, val_pos_s, val_neg_m, val_neg_s

# Get upper and lower boundary for the predicted distances
lower_lim = max(MARGIN - 2.0 * (MARGIN - margin_pos), 0.)
upper_lim = min(MARGIN + 2.0 * (margin_neg - MARGIN), 2.)
   
def dist_to_prob(predictions):
    y_prob = 1 - (np.clip(predictions, lower_lim, upper_lim) - lower_lim) / (upper_lim - lower_lim)
    return y_prob
  
class MetricCallback(keras.callbacks.Callback):
    def __init__(self, logdir):
        super(Callback, self).__init__()
        if not os.path.exists(logdir):
            os.makedirs(logdir)
        self.train_writer = tf.summary.create_file_writer(logdir + '/train')
        self.valid_writer = tf.summary.create_file_writer(logdir + '/valid')
        self.class_encoded = {
            0: 'not_related',
            1: 'related'
        }
        
    def tb_writer(self, items_to_write, wtype, epoch):
        writer = self.train_writer if wtype == 'train' else self.valid_writer
        
        with writer.as_default():
            for name, value in items_to_write.items():
                tf.summary.scalar(name, value, epoch)
            writer.flush()
        
    def on_epoch_end(self, epoch, logs={}):
        val_true = []
        val_pred = []
        batches = batched_pairs(
            val_rlts,
            batch_size,
            eval_dataset_period,
            input_shape,
            preprocess,
            shuffle=False,
            slice_imgs=2
        )
        
        for batch in batches:
            pred = self.model.predict(batch[0])
            val_pred.append(pred)
            val_true.extend(list(batch[1]))
        
        val_true = np.array(val_true)
        val_pred = np.concatenate(val_pred, axis=0).squeeze()
        val_loss = contrastive_loss(K.constant(val_true), K.constant(val_pred))
        
        val_true = val_true.astype(int)
        val_pos_m, val_pos_s, val_neg_m, val_neg_s = val_distance_stats(val_pred, val_true)
        threshold = MARGIN
        
        # Precision and recall
        val_cls = (val_pred < threshold).astype(int)
        val_precision, val_recall, _, _ = precision_recall_fscore_support(val_true, val_cls, labels=[0, 1])
        val_accuracy = accuracy_score(val_true, val_cls)
        
        # Area under ROC
        val_probs = dist_to_prob(val_pred)
        val_roc_auc = roc_auc_score(val_true, val_probs)
        train_loss = logs['loss']
        tb_logs = {}
        tb_logs['train/loss'] = train_loss
        
        self.tb_writer(tb_logs, wtype='train', epoch=epoch)
        
        tb_logs = {}
        tb_logs['valid/loss'] = val_loss
        logs['val_loss'] = val_loss
        for k, v in self.class_encoded.items():
            tb_logs['valid/precision/' + v] = val_precision[k]
            tb_logs['valid/recall/' + v] = val_recall[k]
            tb_logs['valid/dist_mean/' + v] = val_pos_m if k else val_neg_m
            tb_logs['valid/dist_std/' + v] = val_pos_s if k else val_neg_s
        
        tb_logs['valid/accuracy'] = val_accuracy
        logs['val_roc_auc'] = val_roc_auc
        tb_logs['valid/roc_auc'] = val_roc_auc

        self.tb_writer(tb_logs, wtype='valid', epoch=epoch)

In [ ]:
optimizer = eval(optimizer)(learning_rate=learning_rate)
model.compile(loss=[contrastive_loss, contrastive_loss], optimizer=optimizer)
model.summary()

In [ ]:
model_name = 'facenet_M_0.7_V_1.0_pos_1.6_nol2_adam_cos_001_freeze-after-add_10'

#Save training configuration
with open(f'configs/{model_name}.json', 'w') as f:
    json.dump(train_config, f)

logdir = os.path.join('logs', model_name)
ckpt_dir = os.path.join('checkpoints', model_name)
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)
ckpt_callback = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(ckpt_dir, 'model.hdf5'),
    monitor='val_roc_auc',
    save_best_only=True,
    mode='max',
    save_weights_only=True,
    verbose=1,
)
metric_callback = MetricCallback(logdir)

In [ ]:
def repeat_generator(rlts, batch_size, dataset_period, input_shape, preprocess):
    while True:
        for e in batched_pairs(rlts, batch_size, dataset_period, input_shape, preprocess):
            yield e
            
model.fit(
    repeat_generator(train_rlts, batch_size, dataset_period, input_shape, preprocess),
    epochs=epochs,
    steps_per_epoch=train_len//batch_size,
    callbacks=[metric_callback, ckpt_callback]
)

# Submission

In [ ]:
# Load submission pairs
submission_path = 'data/sample_submission.csv'
submission_df = pd.read_csv(submission_path)

In [ ]:
# Load models
ckpt_path = 'checkpoints/vggface_res50_M_0.7_V_1.0_pos_1.6_2xconv1d-noactiv-nol2_2048_2048_adam_euc_cos_014_freeze-after-add_10/model.hdf5'
model.load_weights(ckpt_path)
# model = keras.models.load_model(ckpt_path, custom_objects={'contrastive_loss': contrastive_loss})

In [ ]:
# Iterate over submission pairs
submission_df = submission_df.astype({'is_related': 'float'})
is_related = submission_df['is_related']
predictions = []
for idx, row in submission_df.iterrows():
    # Load images
    img_pair = row['img_pair']
    img1_name, img2_name = img_pair.split('-')
    img1_path = os.path.join('data/test', img1_name)
    img2_path = os.path.join('data/test', img2_name)
    img1 = image.load_img(img1_path)
    img2 = image.load_img(img2_path)
    img1 = preprocess(np.array(img1).astype('float32'))
    img2 = preprocess(np.array(img2).astype('float32'))
    img1 = np.expand_dims(img1, 0)
    img2 = np.expand_dims(img2, 0)
    
    # Do an inference, and calculate probability according to distance
    y_pred1, y_pred2 = model.predict({'input_1':img1, 'input_2':img2})
    y_pred1 = y_pred1.squeeze()
    y_pred2 = y_pred2.squeeze()
    y_prob1 = dist_to_prob(y_pred1)
    y_prob2 = dist_to_prob(y_pred2)
    y_prob = 0.5 * (y_prob1 + y_prob2)
    
    predictions.append(y_prob)
    is_related[idx] = y_prob
    
    # Print step
    if idx % 100 == 0:
        print(f'Processed rows: {idx}')
        
submission_df.to_csv(f'submission.csv', index=False)

In [ ]:
plt.subplots(figsize=(30, 8))
plt.hist(predictions, 1000)
plt.locator_params(axis='x', nbins=100)
plt.xticks(rotation = 45)
plt.show()

In [ ]:
submission_df = pd.read_csv(submission_path)
is_related = submission_df['is_related']
# print(is_related.sum())
for i, pred in enumerate(predictions):
    if pred < 0.149:
        is_related[i] = 1
submission_df.to_csv(f'submission.csv', index=False)